In [1]:
import numpy as np
import pandas as pd
import sklearn.model_selection
import gc

In [2]:
'''
# my previous work
train = reload()

# label: winPlacePerc
X = train.drop(["winPlacePerc"],axis =1)
Y = train["winPlacePerc"]
print(set(X["matchType"]))

# obviously, match_type: 'duo-fpp' should be one hot encoding
# {'normal-squad', 'duo-fpp', 'solo', 'crashfpp', 'normal-duo', 'flarefpp', 'normal-solo', 'normal-squad-fpp', 
# 'normal-duoThe -fpp', 'duo', 'normal-solo-fpp', 'crashtpp', 'squad-fpp', 'flaretpp', 'squad', 'solo-fpp'}
matchType = ['normal-squad', 'duo-fpp', 'solo', 'crashfpp', 'normal-duo', 'flarefpp', 'normal-solo', 'normal-squad-fpp','normal-duo-fpp', 'duo', 'normal-solo-fpp', 'crashtpp', 'squad-fpp', 'flaretpp', 'squad', 'solo-fpp']
for match in matchType:
    X[match] = X["matchType"] == match
X = X.drop(["matchType"],axis=1)
print(X.columns.values)

# So I do not use ID, groupId, matchId here
X = X.drop(["Id","groupId","matchId"],axis=1)

pd.options.display.max_columns = None
print(X[:5])

# a little feature enhance try
# "recover"
X["recover"] = X["heals"] + X["boosts"]

print(X.columns.values[:25])
print(X.columns.values[-1:])
print(X.columns.values[25:-1])
# no normalization for one hot encoding

X = X.values
Y = Y.values

def std_norm(arr):
    arr = (arr - np.mean(arr))/ np.std(arr)
    return arr
    
X = X.T
for i in range(25):
    X[i] = std_norm(X[i])
X[-1] = std_norm(X[-1])
X = X.T


'''

'\n# my previous work\ntrain = reload()\n\n# label: winPlacePerc\nX = train.drop(["winPlacePerc"],axis =1)\nY = train["winPlacePerc"]\nprint(set(X["matchType"]))\n\n# obviously, match_type: \'duo-fpp\' should be one hot encoding\n# {\'normal-squad\', \'duo-fpp\', \'solo\', \'crashfpp\', \'normal-duo\', \'flarefpp\', \'normal-solo\', \'normal-squad-fpp\', \n# \'normal-duoThe -fpp\', \'duo\', \'normal-solo-fpp\', \'crashtpp\', \'squad-fpp\', \'flaretpp\', \'squad\', \'solo-fpp\'}\nmatchType = [\'normal-squad\', \'duo-fpp\', \'solo\', \'crashfpp\', \'normal-duo\', \'flarefpp\', \'normal-solo\', \'normal-squad-fpp\',\'normal-duo-fpp\', \'duo\', \'normal-solo-fpp\', \'crashtpp\', \'squad-fpp\', \'flaretpp\', \'squad\', \'solo-fpp\']\nfor match in matchType:\n    X[match] = X["matchType"] == match\nX = X.drop(["matchType"],axis=1)\nprint(X.columns.values)\n\n# So I do not use ID, groupId, matchId here\nX = X.drop(["Id","groupId","matchId"],axis=1)\n\npd.options.display.max_columns = None\npr

In [3]:
#learn from rejasupotaro
def reload():
    gc.collect()
    train = pd.read_csv('./data/train_V2.csv')
    # isna() isin()
    invalid_match_ids = train[train['winPlacePerc'].isna()]['matchId'].values
    train = train[-train['matchId'].isin(invalid_match_ids)]
    return train

In [4]:
# split for new matchId
def train_test_split(df, test_size=0.1):
    match_ids = df['matchId'].unique().tolist()
    train_size = int(len(match_ids) * (1-test_size))
    train_match_ids = random.sample(match_ids,train_size)
    train = df[df['matchId'].isin(train_match_ids)]
    test = df[df['matchId'].isin(train_match_ids)]
    return train,test

idx = np.random.choice(len(X),len(X),replace= False)

X = X[idx]
Y = Y[idx]

In [14]:
# a little experiment
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import time
import random
random.seed(233)

def run_experiment(preprocess):
    df = reload()
    # Inplace: if true, do operation inplace and return None
    df.drop(columns=['matchType'],inplace = True)
    df = preprocess(df)
    
    target = 'winPlacePerc'
    
    cols_to_drop = ['Id','groupId','matchId',target]
    cols_to_fit = [col for col in df.columns if col not in cols_to_drop]
    
    train, val = train_test_split(df,0.1)
    
    model = LinearRegression()
    # no one hot yet
    model.fit(train[cols_to_fit],train[target])
    
    y_true = val[target]
    y_pred = model.predict(val[cols_to_fit])
    
    return mean_absolute_error(y_true,y_pred)

def run_experiments(preprocesses):
    results = []
    for preprocess in preprocesses:
        start = time.time()
        score = run_experiment(preprocess)
        execution_time = time.time() - start
        results.append({
            'name': preprocess.__name__,
            'score':score,
            'execution time': f'{round(execution_time,2)}s'
        })
        gc.collect()
        
    return pd.DataFrame(results,columns=['name','score','exectution time']).sort_values(by='score')
    
    
    

In [23]:
def original(df):
    return df

def recover(df):
    df['recover'] = df['heals'] + df['boosts']
    return df
    
def players_in_team(df):
    # take the num to players in current team into consideration, new feature
    agg = df.groupby(['groupId']).size().to_frame('player_in_team')
    # wait check what it looks like
    res =  df.merge(agg,how='left',on=['groupId'])
    print('#player in team',res.columns.values)
    return res
    
def total_distance(df):
    df['total_distance'] = df['rideDistance'] + df['swimDistance']+ df['walkDistance']
    return df

def headshotKills_over_kills(df):
    df['headshotKills_over_kills'] = df['headshotKills'] / df['kills']
    df['headshotKills_over_kills'].fillna(0,inplace=True)
    return df
    
def killPlace_over_maxPlace(df):
    df['killPlace_over_maxPlace'] = df['killPlace'] / df['maxPlace']
    df['killPlace_over_maxPlace'].fillna(0, inplace=True)
    df['killPlace_over_maxPlace'].replace(np.inf, 0, inplace=True)
    return df

def walkDistance_over_heals(df):
    df['walkDistance_over_heals'] = df['walkDistance'] / df['heals']
    df['walkDistance_over_heals'].fillna(0, inplace=True)
    df['walkDistance_over_heals'].replace(np.inf, 0, inplace=True)
    return df

def walkDistance_over_kills(df):
    df['walkDistance_over_kills'] = df['walkDistance'] / df['kills']
    df['walkDistance_over_kills'].fillna(0, inplace=True)
    df['walkDistance_over_kills'].replace(np.inf, 0, inplace=True)
    return df

def teamwork(df):
    df['teamwork'] = df['assists'] + df['revives']
    return df


In [27]:
run_experiments([
    original,
    recover,
    players_in_team,
    total_distance,
    headshotKills_over_kills,
    killPlace_over_maxPlace,
    walkDistance_over_heals,
    walkDistance_over_kills,
    teamwork
])

KeyboardInterrupt: 

In [28]:
start = time.time()
k = 0
for _ in range(100000):
    k += 1
print(time.time() - start)

0.013187170028686523


In [9]:

# permutation importance: random forest https://www.stat.berkeley.edu/~breiman/randomforest2001.pdf

In [10]:
#SHAP values

OSError: dlopen(/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/lightgbm/lib_lightgbm.so, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/lightgbm/lib_lightgbm.so
  Reason: image not found